In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#            Machine Learning y Deep Learning
# ----------------------------------------------------------
#         Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#           Redes neuronales profundas LSTM
# ==========================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# ---------------------- Cargando datos ----------------------------------------
# Cargar el conjunto de datos:
url =  "https://raw.githubusercontent.com/rogon666/UMSA/main/AIMLDL/Datos/BTCUSD.csv"
# Cargar los datos en un DataFrame
data= pd.read_csv(url)
# Mostrar las primeras filas del DataFrame
print(data.head())

In [ ]:
# Preparacion de datos
# Preparar el objetivo (y)
y = data['Close'].values.reshape(-1, 1)

# Crear la matriz de características X como rezagos de y
def create_lagged_features(y, lag=1):
    X = np.zeros((len(y) - lag, lag))
    for i in range(lag):
        X[:, i] = y[(lag - i - 1):(len(y) - i - 1), 0]
    y_lagged = y[lag:]
    return X, y_lagged

# Usar rezagos para las características
lag = 60
X, y_lagged = create_lagged_features(y, lag)

# Escalar los datos
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X = scaler_X.fit_transform(X)
y_lagged = scaler_y.fit_transform(y_lagged)

# Reshape para que sea compatible con LSTM/GRU (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Partición de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y_lagged, test_size=0.15, shuffle=False)

In [ ]:
# Definir el modelo LSTM profundo
deepLSTM = Sequential()
# Capa de entrada LSTM, regularizada con drop-out
deepLSTM.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
deepLSTM.add(Dropout(0.3))
# Segunda capa LSTM, regularizada con una penalidad L2
deepLSTM.add(LSTM(units=128, return_sequences=True, kernel_regularizer=l2(0.01)))
# Tercera capa LSTM (sin return_sequences porque es la última capa LSTM), regularizada con drop-out
deepLSTM.add(LSTM(units=64))
deepLSTM.add(Dropout(0.3))
# Capa densa despues de LSTM:
deepLSTM.add(Dense(64, activation='relu'))  # Capa densa después de las LSTM
# Capa densa de salida:
deepLSTM.add(Dense(1))  # Capa de salida

# Compilación del modelo
deepLSTM.compile(optimizer='adam',
                 loss='mean_squared_error')

# Entrenamiento del modelo
deepLSTM.fit(X_train, y_train, epochs=10)

In [ ]:
# Predicción con el modelo LSTM profundo
y_pred_train_lstm = deepLSTM.predict(X_train)
y_pred_test_lstm = deepLSTM.predict(X_test)

# Desnormalizar para evaluar correctamente
y_train_actual = scaler_y.inverse_transform(y_train)
y_test_actual = scaler_y.inverse_transform(y_test)
y_pred_train_lstm_actual = scaler_y.inverse_transform(y_pred_train_lstm)
y_pred_test_lstm_actual = scaler_y.inverse_transform(y_pred_test_lstm)

# Evaluación del modelo LSTM
mae_train_lstm = mean_absolute_error(y_train_actual, y_pred_train_lstm_actual)
rmse_train_lstm = np.sqrt(mean_squared_error(y_train_actual, y_pred_train_lstm_actual))
mae_test_lstm = mean_absolute_error(y_test_actual, y_pred_test_lstm_actual)
rmse_test_lstm = np.sqrt(mean_squared_error(y_test_actual, y_pred_test_lstm_actual))

print(f"LSTM MAE (train): {mae_train_lstm}")
print(f"LSTM RMSE (train): {rmse_train_lstm}")
print(f"LSTM MAE (test): {mae_test_lstm}")
print(f"LSTM RMSE (test): {rmse_test_lstm}")

# Gráficos de comparación
plt.figure(figsize=(7, 7))

# Predicciones vs Realidad para LSTM
plt.plot(y_test_actual, label='Valores observados')
plt.plot(y_pred_test_lstm_actual, label='Valores predichos LSTM')
plt.title('Comparación entre los precios reales y predichos (LSTM)')
plt.xlabel('Días')
plt.ylabel('Precio Bitcoin (USD)')
plt.legend()